## Imports Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, utils, losses, optimizers, initializers, regularizers
from keras.wrappers import scikit_learn
from keras.models import *
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils, image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

import random
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Reads the `txt` File Specifying the Image File Locations

In [2]:
# flist = list(pd.read_csv('flist.txt', header = None)[0])
flist = list(pd.read_csv('/kaggle/input/dissertation-1-data/Files/flist.txt', header = None)[0])

## Overview of the `properties` Dataset

Read the `properties` dataset first, and make sure that `property type` is a categorical variable.

In [3]:
# properties = pd.read_csv('properties.csv')
# properties_juny12 = pd.read_csv('properties_juny12.csv')
properties = pd.read_csv('/kaggle/input/dissertation-1-data/Files/properties.csv')
properties_juny12 = pd.read_csv('/kaggle/input/dissertation-1-data/Files/properties_juny12.csv')

properties_full = pd.concat([properties, properties_juny12])
properties = properties_full
properties.propertyType = properties.propertyType.astype('category')
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37402 entries, 0 to 19851
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Unnamed: 0    37402 non-null  int64   
 1   address       37402 non-null  object  
 2   propertyType  37402 non-null  category
 3   bedrooms      24486 non-null  float64 
 4   detailUrl     37402 non-null  object  
 5   location_lat  37402 non-null  float64 
 6   location_lng  37402 non-null  float64 
 7   property_id   37402 non-null  object  
dtypes: category(1), float64(3), int64(1), object(3)
memory usage: 2.3+ MB


## A Subset of the `properties` Dataset

As random samples of images have been obtained previously, a subset of the whole `properties` dataset could hence be formulated by selecting the rows of the whole `properties` dataset corresponding to the selected samples.

In [4]:
flist_id = list(map(lambda string: string[-40 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
properties_sub = properties_sub.drop_duplicates(['location_lat', 'location_lng'])

# properties_sub = pd.read_csv('properties_sub.csv')
# properties_sub = pd.read_csv('/kaggle/input/dissertation-1-data/Files/properties_sub.csv')

properties_sub.propertyType = properties_sub.propertyType.astype('category')
flist_id = list(properties_sub.property_id)
properties_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15484 entries, 0 to 19851
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Unnamed: 0    15484 non-null  int64   
 1   address       15484 non-null  object  
 2   propertyType  15484 non-null  category
 3   bedrooms      10967 non-null  float64 
 4   detailUrl     15484 non-null  object  
 5   location_lat  15484 non-null  float64 
 6   location_lng  15484 non-null  float64 
 7   property_id   15484 non-null  object  
dtypes: category(1), float64(3), int64(1), object(3)
memory usage: 983.1+ KB


In [5]:
properties_sub.propertyType.value_counts(sort = False)

Detached         3337
Flat             2252
Semi-Detached    4062
Terraced         4124
Unknown          1709
Name: propertyType, dtype: int64

The original data should be splitted into training and testing sets, and the testing set contains 30% of the original data.

In [6]:
# directory = 'street_view/'
directory = '/kaggle/input/dissertation-1-data/Files/street_view/'

height = 64
width = 64
batch = 32
seed = np.random.randint(1, np.power(2, 32) - 1)

training = image_dataset_from_directory(
  directory,
  validation_split = 0.3,
  subset = 'training',
  seed = seed,
  image_size = (height, width),
  batch_size = batch,
  label_mode = 'categorical')

validation = image_dataset_from_directory(
  directory,
  validation_split = 0.3,
  subset = 'validation',
  seed = seed,
  image_size = (height, width),
  batch_size = batch,
  label_mode = 'categorical')

training = training.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
validation = validation.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

Found 13775 files belonging to 4 classes.
Using 9643 files for training.
Found 13775 files belonging to 4 classes.
Using 4132 files for validation.


In [7]:
propertyType_train_fac = np.argmax(np.asarray(list(training.unbatch().map(lambda x, y: y))), axis = 1)
propertyType_validation_fac = np.argmax(np.asarray(list(validation.unbatch().map(lambda x, y: y))), axis = 1)
labels = pd.Series(propertyType_train_fac).astype('category')

In [8]:
loss = losses.CategoricalCrossentropy()
weights = sum(labels.value_counts()) / (labels.value_counts(sort = False) * len(labels.cat.categories))
loss.weighted = weights

In [ ]:
# sample_weights = weights[propertyType_train_fac]
# training_new = training.unbatch().batch(1).map(lambda x, y: (x, y, sample_weights))

In [9]:
Callbacks = [
             EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 8, restore_best_weights = True), 
             ReduceLROnPlateau(monitor = 'val_accuracy', factor = 1e-2, patience = 0, cooldown = 0)
            ]

## Multi-Class Classification Using Neural Network

### Multi-Layer Perceptron (MLP) model

In [10]:
mlp = Sequential([
                  Rescaling(1. / 255, input_shape = (height, width, 3)),
                  Flatten(),
                  Dense(128, activation = tf.nn.leaky_relu),
                  Dense(
                        len(labels.cat.categories), 
                        activation = tf.nn.softmax, 
                        kernel_initializer = initializers.RandomNormal(),
                        bias_initializer = initializers.Zeros(),
                        kernel_regularizer = regularizers.L1(1e-4),
                        bias_regularizer = regularizers.L1(1),
                        activity_regularizer = regularizers.L1(1)
                       )
                ])
mlp.compile(loss = loss, optimizer = optimizers.Adam(), metrics = ['accuracy'])
mlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 128)               1572992   
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 1,573,508
Trainable params: 1,573,508
Non-trainable params: 0
_________________________________________________________________


In [11]:
mlp.fit(training, validation_data = validation, epochs = 64, batch_size = 64, callbacks = Callbacks)

Epoch 1/64
302/302 [==============================] - 5s 6ms/step - loss: 2.5337 - accuracy: 0.3317 - val_loss: 2.3079 - val_accuracy: 0.3572 - lr: 0.0010
Epoch 2/64
302/302 [==============================] - 1s 4ms/step - loss: 2.3339 - accuracy: 0.3706 - val_loss: 2.3113 - val_accuracy: 0.3722 - lr: 0.0010
Epoch 3/64
302/302 [==============================] - 1s 4ms/step - loss: 2.2967 - accuracy: 0.3851 - val_loss: 2.3173 - val_accuracy: 0.3405 - lr: 0.0010
Epoch 4/64
302/302 [==============================] - 2s 5ms/step - loss: 2.2247 - accuracy: 0.4540 - val_loss: 2.2648 - val_accuracy: 0.4046 - lr: 1.0000e-05
Epoch 5/64
302/302 [==============================] - 2s 5ms/step - loss: 2.2194 - accuracy: 0.4539 - val_loss: 2.2640 - val_accuracy: 0.4030 - lr: 1.0000e-05
Epoch 6/64
302/302 [==============================] - 1s 4ms/step - loss: 2.2169 - accuracy: 0.4555 - val_loss: 2.2639 - val_accuracy: 0.4030 - lr: 1.0000e-07
Epoch 7/64
302/302 [==============================] - 2s 6

In [12]:
propertyType_validation_pred_MLP = np.argmax(mlp.predict(validation), axis = 1)
confusion_matrix(propertyType_validation_fac, propertyType_validation_pred_MLP)

130/130 [==============================] - 0s 2ms/step


array([[232,  68, 467, 202],
       [ 50, 212, 146, 259],
       [154,  70, 636, 358],
       [116, 135, 435, 592]])

In [13]:
print(
      classification_report(
                      propertyType_validation_fac, 
                      propertyType_validation_pred_MLP, 
                      target_names = ['Detached', 'Flat', 'Semi-Detached', 'Terraced']
      )
)

               precision    recall  f1-score   support

     Detached       0.42      0.24      0.31       969
         Flat       0.44      0.32      0.37       667
Semi-Detached       0.38      0.52      0.44      1218
     Terraced       0.42      0.46      0.44      1278

     accuracy                           0.40      4132
    macro avg       0.41      0.39      0.39      4132
 weighted avg       0.41      0.40      0.40      4132



## Convolutional Neural Network (CNN) Model

In [14]:
cnn = Sequential([
                  Rescaling(1. / 255, input_shape = (height, width, 3)),
                  Conv2D(8, 4, padding = 'same', activation = tf.nn.leaky_relu),
                  MaxPooling2D(pool_size = (2, 2)),
                  Conv2D(4, 4, padding = 'same', activation = tf.nn.leaky_relu),
                  MaxPooling2D(pool_size = (2, 2)),
                  Flatten(),
                  Dense(16, activation = tf.nn.leaky_relu),
                  Dropout(.25),
                  Dense(
                        len(labels.cat.categories), 
                        activation = tf.nn.softmax, 
                        kernel_initializer = initializers.RandomNormal(),
                        bias_initializer = initializers.Zeros(),
                        kernel_regularizer = regularizers.L1(1e-4),
                        bias_regularizer = regularizers.L1(1),
                        activity_regularizer = regularizers.L1(1)
                       )
                ])
cnn.compile(loss = loss, optimizer = optimizers.Adam(), metrics = ['accuracy'])
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 8)         392       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 4)         516       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1024)             

In [15]:
cnn.fit(training, validation_data = validation, epochs = 64, batch_size = 64, callbacks = Callbacks)

Epoch 1/64
302/302 [==============================] - 8s 7ms/step - loss: 2.3302 - accuracy: 0.3397 - val_loss: 2.2762 - val_accuracy: 0.3950 - lr: 0.0010
Epoch 2/64
302/302 [==============================] - 2s 6ms/step - loss: 2.2751 - accuracy: 0.3968 - val_loss: 2.2557 - val_accuracy: 0.4107 - lr: 0.0010
Epoch 3/64
302/302 [==============================] - 2s 6ms/step - loss: 2.2622 - accuracy: 0.4066 - val_loss: 2.2455 - val_accuracy: 0.4121 - lr: 0.0010
Epoch 4/64
302/302 [==============================] - 2s 6ms/step - loss: 2.2506 - accuracy: 0.4183 - val_loss: 2.2320 - val_accuracy: 0.4293 - lr: 0.0010
Epoch 5/64
302/302 [==============================] - 2s 6ms/step - loss: 2.2385 - accuracy: 0.4292 - val_loss: 2.2251 - val_accuracy: 0.4388 - lr: 0.0010
Epoch 6/64
302/302 [==============================] - 2s 8ms/step - loss: 2.2265 - accuracy: 0.4353 - val_loss: 2.2240 - val_accuracy: 0.4395 - lr: 0.0010
Epoch 7/64
302/302 [==============================] - 2s 7ms/step - lo

In [16]:
propertyType_validation_pred_CNN = np.argmax(cnn.predict(validation), axis = 1)
confusion_matrix(propertyType_validation_fac, propertyType_validation_pred_CNN)

130/130 [==============================] - 2s 2ms/step


array([[376,  57, 370, 166],
       [ 93, 229, 113, 232],
       [253,  58, 624, 283],
       [177, 123, 369, 609]])

In [17]:
print(
      classification_report(
                      propertyType_validation_fac, 
                      propertyType_validation_pred_CNN, 
                      target_names = ['Detached', 'Flat', 'Semi-Detached', 'Terraced']
      )
)

               precision    recall  f1-score   support

     Detached       0.42      0.39      0.40       969
         Flat       0.49      0.34      0.40       667
Semi-Detached       0.42      0.51      0.46      1218
     Terraced       0.47      0.48      0.47      1278

     accuracy                           0.44      4132
    macro avg       0.45      0.43      0.44      4132
 weighted avg       0.45      0.44      0.44      4132



In [18]:
cnn2 = Sequential([
                  Rescaling(1. / 255, input_shape = (height, width, 3)),
                  Conv2D(50, 4, padding = 'same', activation = tf.nn.leaky_relu),
                  MaxPooling2D(pool_size = (2, 2)),
                  Conv2D(40, 4, padding = 'same', activation = tf.nn.leaky_relu),
                  MaxPooling2D(pool_size = (2, 2)),
                  Flatten(),
                  Dense(40, activation = tf.nn.leaky_relu),
                  Dropout(.25),
                  Dense(
                        len(labels.cat.categories), 
                        activation = tf.nn.softmax, 
                        kernel_initializer = initializers.RandomNormal(),
                        bias_initializer = initializers.Zeros(),
                        kernel_regularizer = regularizers.L1(1e-4),
                        bias_regularizer = regularizers.L1(1),
                        activity_regularizer = regularizers.L1(1)
                       )
                ])
cnn2.compile(loss = loss, optimizer = optimizers.Adam(), metrics = ['accuracy'])
cnn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 50)        2450      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 50)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 40)        32040     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 40)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 10240)            

In [19]:
cnn2.fit(training, validation_data = validation, epochs = 64, batch_size = 64, callbacks = Callbacks)

Epoch 1/64
302/302 [==============================] - 4s 11ms/step - loss: 2.3023 - accuracy: 0.3620 - val_loss: 2.2505 - val_accuracy: 0.4105 - lr: 0.0010
Epoch 2/64
302/302 [==============================] - 3s 10ms/step - loss: 2.2529 - accuracy: 0.4165 - val_loss: 2.2353 - val_accuracy: 0.4279 - lr: 0.0010
Epoch 3/64
302/302 [==============================] - 3s 10ms/step - loss: 2.2269 - accuracy: 0.4386 - val_loss: 2.2264 - val_accuracy: 0.4337 - lr: 0.0010
Epoch 4/64
302/302 [==============================] - 3s 10ms/step - loss: 2.2042 - accuracy: 0.4465 - val_loss: 2.2230 - val_accuracy: 0.4429 - lr: 0.0010
Epoch 5/64
302/302 [==============================] - 3s 10ms/step - loss: 2.1747 - accuracy: 0.4666 - val_loss: 2.2353 - val_accuracy: 0.4315 - lr: 0.0010
Epoch 6/64
302/302 [==============================] - 3s 10ms/step - loss: 2.1177 - accuracy: 0.5040 - val_loss: 2.2187 - val_accuracy: 0.4441 - lr: 1.0000e-05
Epoch 7/64
302/302 [==============================] - 3s 10m

In [20]:
propertyType_validation_pred_CNN2 = np.argmax(cnn2.predict(validation), axis = 1)
confusion_matrix(propertyType_validation_fac, propertyType_validation_pred_CNN2)

130/130 [==============================] - 1s 4ms/step


array([[378,  66, 384, 141],
       [ 96, 269, 134, 168],
       [254,  74, 640, 250],
       [174, 119, 408, 577]])

In [21]:
print(
      classification_report(
                      propertyType_validation_fac, 
                      propertyType_validation_pred_CNN2, 
                      target_names = ['Detached', 'Flat', 'Semi-Detached', 'Terraced']
      )
)

               precision    recall  f1-score   support

     Detached       0.42      0.39      0.40       969
         Flat       0.51      0.40      0.45       667
Semi-Detached       0.41      0.53      0.46      1218
     Terraced       0.51      0.45      0.48      1278

     accuracy                           0.45      4132
    macro avg       0.46      0.44      0.45      4132
 weighted avg       0.46      0.45      0.45      4132

